In [1]:
import os
import numpy as np
from PIL import Image

def npy_to_tif(input_folder, output_folder):
    """
    Converts all .npy files in the input folder to .tif images and saves them in the output folder.

    Args:
        input_folder (str): Path to the folder containing .npy files.
        output_folder (str): Path to the folder where .tif images will be saved.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get a list of all .npy files in the input folder
    npy_files = [f for f in os.listdir(input_folder) if f.endswith('.npy')]

    if not npy_files:
        print("No .npy files found in the input folder.")
        return

    print(f"Found {len(npy_files)} .npy files. Processing...")

    for npy_file in npy_files:
        # Construct the full file paths
        npy_path = os.path.join(input_folder, npy_file)
        tif_filename = os.path.splitext(npy_file)[0] + ".tif"
        tif_path = os.path.join(output_folder, tif_filename)

        try:
            # Load the .npy array
            data = np.load(npy_path)

            # Normalize the data to fit into 8-bit grayscale
            normalized_data = ((data - np.min(data)) / (np.max(data) - np.min(data)) * 255).astype(np.uint8)

            # Convert to a PIL image and save as .tif
            image = Image.fromarray(normalized_data)
            image.save(tif_path)

            print(f"Converted {npy_file} to {tif_filename}.")
        except Exception as e:
            print(f"Error processing {npy_file}: {e}")

    print("Conversion complete.")

# Example usage
input_folder = "/home/ABTLUS/victor.secco/data_processing/HRTEM_software_rodrigo/HRTEM crops 640x640"  # Replace with the path to your folder containing .npy files
output_folder = "/home/ABTLUS/victor.secco/data_processing/HRTEM_software_rodrigo/HRTEM crops 640x640 tif"   # Replace with the path to your output folder
npy_to_tif(input_folder, output_folder)


Found 13248 .npy files. Processing...
Converted HRTEM_0075-038.npy to HRTEM_0075-038.tif.
Converted HRTEM_0095-050.npy to HRTEM_0095-050.tif.
Converted HRTEM_0152-052.npy to HRTEM_0152-052.tif.
Converted HRTEM_0083-002.npy to HRTEM_0083-002.tif.
Converted HRTEM_0166-051.npy to HRTEM_0166-051.tif.
Converted HRTEM_0050-027.npy to HRTEM_0050-027.tif.
Converted HRTEM_0085-024.npy to HRTEM_0085-024.tif.
Converted HRTEM_0132-086.npy to HRTEM_0132-086.tif.
Converted HRTEM_0133-020.npy to HRTEM_0133-020.tif.
Converted HRTEM_0167-050.npy to HRTEM_0167-050.tif.
Converted HRTEM_0124-014.npy to HRTEM_0124-014.tif.
Converted HRTEM_0050-004.npy to HRTEM_0050-004.tif.
Converted HRTEM_0175-028.npy to HRTEM_0175-028.tif.
Converted HRTEM_0166-029.npy to HRTEM_0166-029.tif.
Converted HRTEM_0012-040.npy to HRTEM_0012-040.tif.
Converted HRTEM_0010-046.npy to HRTEM_0010-046.tif.
Converted HRTEM_0164-019.npy to HRTEM_0164-019.tif.
Converted HRTEM_0105-016.npy to HRTEM_0105-016.tif.
Converted HRTEM_0018-055.n

In [2]:
import cv2
import numpy as np
import hyperspy.api as hs

file_path = '/home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0002.emd'

data = hs.load(file_path)
img = np.array(data)

img = img/img.max()
img = img*255

cv2.imshow('Grayscale Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('/home/ABTLUS/victor.secco/data_processing/Imagens/CdSeS2-_image.png', img)


True

In [ ]:
directory = '/home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/Ilum/2025/17fev/'
names = [x for x in os.listdir(directory) if x.endswith('.ser')] 

file_path = os.path.join(directory, names[0])
data = hs.load(file_path, signal_type='diffraction', lazy=True)

In [5]:
import os
import hyperspy.api as hs
import numpy as np

# Define the folder containing .ser files
input_folder = "/home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2"
output_folder = "/home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Iterate through all .ser files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".ser"):
        file_path = os.path.join(input_folder, file_name)
        try:
            img = hs.load(file_path, signal_type='diffraction')

            # Check if data is a stack of 2D images
            if img.data.ndim == 3:
                base_name = os.path.splitext(file_name)[0]
                for i in range(img.data.shape[0]):
                    frame = img.inav[i]

                    # Normalize to uint16
                    data = frame.data
                    min_val = np.min(data)
                    max_val = np.max(data)
                    if max_val > min_val:
                        data = ((data - min_val) / (max_val - min_val) * 65535).astype(np.uint16)
                    else:
                        data = np.zeros_like(data, dtype=np.uint16)

                    # Save each frame as an individual TIFF
                    frame_file = os.path.join(output_folder, f"{base_name}_frame{i:02d}.tif")
                    hs.signals.Signal2D(data).save(frame_file, overwrite=True, extension='tiff')
                    print(f"Exported: {frame_file}")
            else:
                print(f"Skipped {file_name}: Not a 3D stack")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")


Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame00.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame01.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame02.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame03.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame04.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame05.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame06.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame07.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1_ROI1_1_frame08.tif
Exported: /home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports/W1

In [9]:
import os
import shutil

# Folder containing the exported .tif images
tif_folder = "/home/ABTLUS/victor.secco/data_processing/ED/CdSe_ePDF/W2/tif_exports"

# Iterate through all .tif files in the folder
for file_name in os.listdir(tif_folder):
    if file_name.endswith(".tiff"):
        # Extract the base name before "_frameXX"
        base_name = file_name.split("_frame")[0]

        # Create subfolder for that base name
        subfolder_path = os.path.join(tif_folder, base_name)
        os.makedirs(subfolder_path, exist_ok=True)

        # Move the .tif file into its subfolder
        src_path = os.path.join(tif_folder, file_name)
        dst_path = os.path.join(subfolder_path, file_name)
        shutil.move(src_path, dst_path)


In [2]:
import os

folder = "/home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM"

def delete_files(folder, extensions):
    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        return
    
    for file_name in os.listdir(folder):
        if file_name.endswith(extensions):
            file_path = os.path.join(folder, file_name)
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")

# Delete all .tif and .png files
delete_files(folder, (".tif", ".tiff"))
delete_files(folder, (".png",))

Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0002.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0004.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 640 kx 0009.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 640 kx 0006.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0003.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0010.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 640 kx 0018.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 640 kx 0011.tiff
Deleted: /home/ABTLUS/victor.secco/data_processing/TEM/CdSe_HRTEM_ILUM/CdSe_HRTEM_ILUM Camera 800 kx 0011.tiff
D